# Imports 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors

import package_DBR
from package_DBR import myRound, SelectPath_RT, Delay_RT, FO_RT, FOPDT, SOPDT,Process, Bode

import package_JM_RS 
from package_JM_RS import LL_RT, Margins, PID_RT, IMC_Tuning

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets


# LL_RT

In [2]:
help(LL_RT)

Help on function LL_RT in module package_JM_RS:

LL_RT(MV, Kp, Tlead, Tlag, Ts, PV, PVInit=0)
    The function "LL_RT" needs to be included in a "for or while loop".
    
    :MV: input vector
    :Kp: process gain
    :Tlead: lead time constant [s]
    :Tlag: lag time constant [s]
    :Ts: sampling period [s]
    :PV: output vector
    :PVInit: (optional: default value is 0)
    
    The function "LL_RT" appends a value to the output vector "PV".
    The appended value is obtained from a recurrent equation.
    The used discretisation method is euler backward difference.



# IMC_Tuning

In [3]:
help(IMC_Tuning)

Help on function IMC_Tuning in module package_JM_RS:

IMC_Tuning(K, Tlag1, Tlag2, theta, gamma=0.5)
    IMC_Tuning(K,Tlag1,TLag2,theta,gamma=0.5)
    This function computes the optimised IMC PID tuning parameters for a SOPDT Process. 
    
    :Kp: process gain
    :Tlag1: first lag time constant [s]
    :Tlag2: second lag time constant [s]
    :theta: delay [s]
    :gamma: used to compute the closed loop time constant TCLP [s] such as TCLP = gamma*T1p with T1p = main time constant of the process. (range for gamma is [0.2 ... 0.9], default value is 0.5)
    
    
    returns (Kc,Ti,Td) that are the PID controller parameters



# PID_RT

In [4]:
help(PID_RT)

Help on function PID_RT in module package_JM_RS:

PID_RT(SP, PV, Man, MV_Man, MV_FF, Kc, Ti, Td, alpha, Ts, MVMin, MVMax, MV, MV_P, MV_I, MV_D, E, ManFF=False, PV_Init=0, E_init=0)
    The function "LL_RT" needs to be included in a "for or while loop".
    Inputs :
    :SP: SP (or SetPoint) vector
    :PV: PV (or Processed Value) vector
    :Man: Man (or Manual controller mode) vector (True or False)
    :MV_Man: MV_Man (or Manual value for MV) vector
    :MV_FF: MV_FF (or FeedForward) vector
    
    
    Parameters : 
    :Kc: controller gain
    :Ti: integral time constant [s]
    :Td: derivative time constant [s]
    :alpha: Tfd = alpha*Td where Tfd is the derivative filter time constant [s]
    :Ts: sampling period [s]
    :MVMin: Minimum value for MV (used for saturation and anti wind-up)
    :MVMax: Maximum value for MV (used for saturation and anti wind-up)
    :ManFF: Activated FF in manual mode (optional: default value is False)
    :PV_Init: Initial value for PV (optional : 

# Input parameters


In [12]:
# Paramètres de simulation

Tsim = 2200
Ts = 1
N = int(Tsim/Ts) + 1

MV0 = 50
DV0 = 50
PV0 = 65

Kp = 0.5194904467673822
T1p = 129.9860279570493
T2p = 2.2419668343920548e-08
thetap = 7.219217782540495

Kd = 0.5066617883318505
T1d = 198.90601996256459
T2d = 8.124468051389487
thetad = 1.0918143519352412

gamma = 0.5

Kc,Ti,Td = IMC_Tuning(Kp,T1p,T2p,thetap)



def f(x) : 
    #1 OLP + no FF
    if x == 1 :
        ManPath = {0 : True,Tsim : True}
        MVManPath = {0 : MV0, Tsim : MV0}
        SPPath = {0 : PV0, 800 : PV0}
        DVPath = {0 : 50, 1000 : 60, Tsim : 60}
        ActivateFF = False
        ManFF = True

    #2 OLP + FF
    if x == 2 :
        ManPath = {0 : True,Tsim : False}
        MVManPath = {0 : MV0, Tsim : MV0}
        SPPath = {0 : PV0, 800 : PV0}
        DVPath = {0 : 50, 1000 : 60, Tsim : 60}
        ActivateFF = True
        ManFF = True

    #3 CLP + no FF
    if x == 3 :
        ManPath = {0 : True, 750: False, Tsim : False}
        MVManPath = {0 : MV0, Tsim : MV0}
        SPPath = {0 : PV0, 800 : PV0-10, Tsim : PV0-10}
        DVPath = {0 : DV0, 1500 : DV0+10, Tsim : DV0+10}
        ActivateFF = False
        ManFF = False
    # Closed loop response to SP change
    if x == 0 :
        ManPath = {0 : False, 750: False, Tsim : False}
        MVManPath = {0 : MV0, Tsim : MV0}
        SPPath = {0 : PV0-10, 650 : PV0,1500:PV0-10, Tsim : PV0-10}
        DVPath = {0 : DV0, 1500 : DV0 , Tsim : DV0 }
        ActivateFF = False
        ManFF = False

    #4 CLP + FF
    if x == 4 : 
        ManPath = {0 : True, 750: False, Tsim : False}
        MVManPath = {0 : MV0, Tsim : MV0}
        SPPath = {0 : PV0, 800 : PV0-10, Tsim : PV0-10}
        DVPath = {0 : DV0, 1500 : DV0+10, Tsim : DV0+10}
        ActivateFF = True
        ManFF = False
    return ManPath, MVManPath, SPPath, DVPath, ActivateFF, ManFF


# Simulation 

In [21]:
def g(x,alpha) : 
    ManPath, MVManPath, SPPath, DVPath, ActivateFF, ManFF = f(x)
    t = []
    SP = [] 
    MV = [] 
    Man = []
    MVMan = []
    MVFF = []
    MVP = []
    MVI = []
    MVD = []
    DV = [] 
    PV = [] 
    E = []

    MVFFDelay = []
    MVFFLL1 = [] 



    MVDelayp = [] 
    PV1p = [] 
    PV2p = []

    MVDelayd = [] 
    PV1d = []
    PV2d = []
    MVMin = 0
    MVMax = 500



    for i in range(0,N): 
        t.append(i*Ts)

        SelectPath_RT(SPPath,t,SP)
        SelectPath_RT(DVPath,t,DV)

        Delay_RT(DV - DV0*np.ones_like(DV),np.max([thetad-thetap,0]),Ts,MVFFDelay)
        LL_RT(MVFFDelay,-Kd/Kp,T1p,T1d,Ts,MVFFLL1)

        if ActivateFF :
            LL_RT(MVFFLL1,1,T2p,T2d,Ts,MVFF)
        else : 
            LL_RT(MVFFLL1,0,T2p,T2d,Ts,MVFF)

        SelectPath_RT(ManPath,t,Man)
        SelectPath_RT(MVManPath,t,MVMan)

        PID_RT(SP,PV,Man,MVMan,MVFF,Kc,Ti,Td,alpha,Ts,MVMin,MVMax,MV,MVP,MVI,MVD,E,ManFF,PV0)

        Delay_RT(MV,thetap,Ts,MVDelayp,MV0)
        FO_RT(MVDelayp,Kp,T1p,Ts,PV1p,0)
        FO_RT(PV1p,1,T2p,Ts,PV2p,0)

        Delay_RT(DV - DV0*np.ones_like(DV),thetad,Ts,MVDelayd,0)
        FO_RT(MVDelayd,Kd,T1d,Ts,PV1d,0)
        FO_RT(PV1d,1,T2d,Ts,PV2d,0)

        PV.append(PV2p[-1] + PV2d[-1] + PV0-Kp*MV0)
    return t, MV, SP, PV, DV, Man





# Plot data

In [22]:

def h(x,alpha) : 
    t, MV, SP, PV, DV, Man = g(x,alpha)
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(4,1)
    fig.set_figheight(22)
    fig.set_figwidth(22)

    l1, = ax1.step([0,t[-1]],[0,100],'k-',linewidth=2,label='Man',where='post')
    ax1.set_ylabel('Value of man [0 or 1]')
    if x == 0 : 
        ax1.set_title('Closed-loop response to a change of SP')
    if x == 1 : 
        ax1.set_title('Open-loop response with PID, no FF')
    if x == 2 : 
        ax1.set_title('Open-loop response with PID and  FF')
    if x == 3 : 
        ax1.set_title('Closed-loop response with PID, no FF')
    if x == 4 : 
        ax1.set_title('Closed-loop response with PID and  FF')
    
    ax1.legend(loc='best')

    l2, = ax2.step([0,t[-1]],[0,100],'b-',linewidth=2,label='MV',where='post')
    ax2.set_ylabel('Value of MV [%]')
    ax2.legend(loc='best')


    l3, = ax3.step([0,t[-1]],[0,100],'k-',linewidth=2,label='SP',where='post')
    l4, = ax3.step([0,t[-1]],[0,100],'g-',linewidth=2,label='PV',where='post')
    ax3.set_ylabel('Value of PV [C°]')
    ax3.legend(loc='best')

    l5, = ax4.step([0,t[-1]],[0,100],'r-',linewidth=2,label='DV',where='post')
    ax4.set_xlabel('Time [s]')
    ax4.set_ylabel('Value of DV [%]')
    ax4.legend(loc='best')

    ManInt = [int(x) for x in Man]
    l1.set_data(t,ManInt)
    l2.set_data(t,MV)
    l3.set_data(t,SP)
    l4.set_data(t,PV)
    l5.set_data(t,DV)

    ax1.set_xlim(0, t[-1]+1)
    ax2.set_xlim(0, t[-1]+1)
    ax3.set_xlim(0, t[-1]+1)
    ax4.set_xlim(0, t[-1]+1)

    ax1.set_ylim(-0.1,1.1)
    ax2.set_ylim(myRound(np.min(MV),5)-5,myRound(np.max(MV),5)+5)
    ax3.set_ylim(myRound(np.min((np.min(PV),np.min(SP))),5)-5, myRound(np.max((np.max(PV),np.max(SP))),5)+5)
    ax4.set_ylim(myRound(np.min(DV),5)-5,myRound(np.max(DV),5)+5)

x = 4
interact(h, x=(0,4), alpha = (0,1,0.1));




# Changer la valeur du slider pour changer de simulation 

interactive(children=(IntSlider(value=2, description='x', max=4), FloatSlider(value=0.0, description='alpha', …